In [124]:
import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

import fastf1 as ff1
import numpy as np
import matplotlib as mpl

from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection

ff1.Cache.enable_cache('./cache') 

In [129]:
# aggregate weather data per race
def agg_weather(weather_data):
    col_agg = {
    'AirTemp':'average',
    'Humidity':'average',
    'Pressure':'average',
    'Rainfall':'max',
    'TrackTemp':'average',
    'WindSpeed':'average'}
    weather_agg = weather_data[['AirTemp','Humidity','Pressure','Rainfall','TrackTemp','WindSpeed']].agg(col_agg,axis=0)
    return weather_agg 

# aggregate lap data according to stints (start/pit stop to pit stop/end)
def agg_tyre(lap_data):
    col_agg = {'TyreLife':'max'}
    lap_agg = lap_data[['Compound','TyreLife','Team','Driver','Stint']].groupby(['Team','Driver','Stint','Compound']).agg(col_agg,axis=0).reset_index()
    return lap_agg 

In [ ]:
%%time
# to scrape
# get number of rounds from event schedule
# for each round filter conventional races (exclude training and sprint), get race session (not practice or qualifying)
# for each session, load session data, process and save aggregated value eg rainfall. no header so need to add yourself
weather_df = pd.DataFrame()

for year in range(2018,2022):
    event_schedule = ff1.get_event_schedule(year)
    event_schedule = event_schedule[event_schedule['EventFormat']=='conventional']
    rounds = event_schedule['RoundNumber']

    for roundnum in rounds:
        print("extracting " + year + " " + roundnum)
        session = ff1.get_session(year,roundnum,'Race')
        session.load()

        # weather
        weather = session.weather_data
        weather_row = pd.DataFrame(agg_weather(weather)).T
        # add identifying columns
        weather_row['round'] = roundnum
        weather_row['year'] = year
        weather_row.to_csv('./data/weather.csv',index=False,header=False,mode='a')

        # tyres
        laps = session.laps
        tyre_row = agg_tyre(laps)
        tyre_row['round'] = roundnum
        tyre_row['year'] = year
        tyre_row.to_csv('./data/tyrelife.csv',index=False,header=False,mode='a')


In [143]:
laps.head()

,Time,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,LapStartTime,Team,Driver,TrackStatus,IsAccurate,LapStartDate
0,0 days 04:19:26.477000,33,NaT,1,2,0 days 04:16:05.434000,NaT,NaT,0 days 00:01:36.992000,0 days 00:00:53.620000,NaT,0 days 04:18:32.857000,0 days 04:19:26.547000,140.0,132.0,159.0,121.0,False,WET,2.0,False,0 days 04:15:59.115000,Red Bull Racing,VER,1,False,2021-08-29 16:17:00.242
1,0 days 04:22:44.826000,33,NaT,2,2,NaT,NaT,0 days 00:00:54.169000,0 days 00:01:32.540000,0 days 00:00:51.665000,0 days 04:20:20.657000,0 days 04:21:53.161000,0 days 04:22:44.922000,178.0,116.0,157.0,129.0,False,WET,3.0,False,0 days 04:19:26.477000,Red Bull Racing,VER,1,False,2021-08-29 16:20:27.604
2,0 days 04:26:08.782000,33,NaT,3,2,NaT,0 days 04:26:04.656000,0 days 00:00:52.264000,0 days 00:01:32.729000,0 days 00:00:59.005000,0 days 04:23:37.097000,0 days 04:25:09.777000,0 days 04:26:09.006000,166.0,110.0,NaN,125.0,False,WET,4.0,False,0 days 04:22:44.826000,Red Bull Racing,VER,1,False,2021-08-29 16:23:45.953
3,0 days 04:19:28.494000,63,NaT,1,2,0 days 04:16:08.396000,NaT,NaT,0 days 00:01:36.790000,0 days 00:00:54.536000,NaT,0 days 04:18:33.958000,0 days 04:19:28.523000,150.0,119.0,162.0,119.0,False,WET,1.0,False,0 days 04:15:59.115000,Williams,RUS,1,False,2021-08-29 16:17:00.242
4,0 days 04:22:47.213000,63,NaT,2,2,NaT,NaT,0 days 00:00:54.706000,0 days 00:01:32.105000,0 days 00:00:51.993000,0 days 04:20:23.115000,0 days 04:21:55.255000,0 days 04:22:47.329000,197.0,120.0,160.0,142.0,False,WET,2.0,False,0 days 04:19:28.494000,Williams,RUS,1,False,2021-08-29 16:20:29.621


In [142]:
event_schedule = ff1.get_event_schedule(2021)
event_schedule.head(3)

,RoundNumber,Country,Location,OfficialEventName,EventDate,EventName,EventFormat,Session1,Session1Date,Session2,Session2Date,Session3,Session3Date,Session4,Session4Date,Session5,Session5Date,F1ApiSupport
0,0,Bahrain,Sakhir,FORMULA 1 ARAMCO PRE-SEASON TESTING 2021,2021-03-14,Pre-Season Test,testing,Practice 1,2021-03-12,Practice 2,2021-03-13,Practice 3,2021-03-14,None,NaT,None,NaT,True
1,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2021,2021-03-28,Bahrain Grand Prix,conventional,Practice 1,2021-03-26,Practice 2,2021-03-26,Practice 3,2021-03-27,Qualifying,2021-03-27,Race,2021-03-28,True
2,2,Italy,Imola,FORMULA 1 PIRELLI GRAN PREMIO DEL MADE IN ITAL...,2021-04-18,Emilia Romagna Grand Prix,conventional,Practice 1,2021-04-16,Practice 2,2021-04-16,Practice 3,2021-04-17,Qualifying,2021-04-17,Race,2021-04-18,True


In [3]:
# read ergast data
circuits = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-09-07/circuits.csv')
constructor_results = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-09-07/constructor_results.csv')
constructor_standings = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-09-07/constructor_standings.csv')
constructors = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-09-07/constructors.csv')
driver_standings = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-09-07/driver_standings.csv')
drivers = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-09-07/drivers.csv')
lap_times = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-09-07/lap_times.csv')
pit_stops = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-09-07/pit_stops.csv')
qualifying = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-09-07/qualifying.csv')
races = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-09-07/races.csv')
results = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-09-07/results.csv')
seasons = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-09-07/seasons.csv')
status = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-09-07/status.csv')